<a href="https://colab.research.google.com/github/wwucla/gpt1/blob/main/gpt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intro
This colab is largely inherent from [Andrej Karpathy's colab](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing#scrollTo=wJpXpmjEYC_T).

Instead of having dev/test code in Colab and build prod code in Python binary (to run on a GPU server), I am putting all code in Colab due to lack of a workstation with GPU.

In [1]:
# @title Imports

!pip install tiktoken
import tiktoken
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00


In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-10-28 17:01:26--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-10-28 17:01:27 (24.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [5]:
# all unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from chars to integers
stoi = {ch : i for i, ch in enumerate(chars)}
itos = {i : ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode('Shakespeare!!!'))
print(decode((encode('Shakespeare!!!'))))

[31, 46, 39, 49, 43, 57, 54, 43, 39, 56, 43, 2, 2, 2]
Shakespeare!!!


In [7]:
## test code: using gpt2 tokenizer
enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab)
print(enc.encode('Hii there'))

50257
[39, 4178, 612]


In [8]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# split 90% data for training, 10% for validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# inspecting data
block_size = 8
train_data[:block_size+1]
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

In [11]:
torch.manual_seed(1234)
batch_size = 4
block_size = 8 # max context length for prediction

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split== 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)
print('----')

for b in range(batch_size): # batch dim
  for t in range(block_size): # time dim
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[51, 59, 57, 58,  1, 39, 61, 39],
        [63,  6,  1, 61, 46, 53,  5, 57],
        [39,  1, 41, 46, 47, 50, 42, 10],
        [53, 59, 10,  0, 37, 53, 59,  1]])
targets
torch.Size([4, 8])
tensor([[59, 57, 58,  1, 39, 61, 39, 63],
        [ 6,  1, 61, 46, 53,  5, 57,  1],
        [ 1, 41, 46, 47, 50, 42, 10,  0],
        [59, 10,  0, 37, 53, 59,  1, 57]])
----
when input is [51] the target: 59
when input is [51, 59] the target: 57
when input is [51, 59, 57] the target: 58
when input is [51, 59, 57, 58] the target: 1
when input is [51, 59, 57, 58, 1] the target: 39
when input is [51, 59, 57, 58, 1, 39] the target: 61
when input is [51, 59, 57, 58, 1, 39, 61] the target: 39
when input is [51, 59, 57, 58, 1, 39, 61, 39] the target: 63
when input is [63] the target: 6
when input is [63, 6] the target: 1
when input is [63, 6, 1] the target: 61
when input is [63, 6, 1, 61] the target: 46
when input is [63, 6, 1, 61, 46] the target: 53
when input is [63, 6, 1